## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [1]:
import numpy as np

In [2]:
A = np.random.randint(0, 1001, size=1_000_000)
A

def f1(A):
    acc, cnt = 0, 0
    for x in A:
        b = x + 100
        acc += b
        cnt += 1
    return acc / cnt

f1(A)

599.84719

In [3]:
%%time
# время выполнения всей ячейки
f1(A)

Wall time: 200 ms


599.84719

In [4]:
%time f1(A) # на уровне 1 строчки

Wall time: 207 ms


599.84719

In [5]:
%%timeit
f1(A)

191 ms ± 9.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
def f2(A):
    acc, cnt = 0, len(A)
    for x in A:
        acc += x
    return acc / cnt + 100

In [7]:
%%time
f2(A)

Wall time: 60 ms


599.84719

In [8]:
def f3(A):
    return np.mean(A) + 100

In [9]:
%%time
f3(A)

Wall time: 0 ns


599.84719

In [10]:
%load_ext line_profiler

In [11]:
%lprun -f f1 f1(A)

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [12]:
import pandas as pd
import string

N = 2_000_000
df = pd.DataFrame(np.random.randn(N, 4), columns=[f"col{i}" for i in range(4)])
df["key"] = np.random.choice(list(string.ascii_letters.lower()), N, replace=True)
df.head(2)

,col0,col1,col2,col3,key
0,0.521787,0.121821,0.469935,-1.722854,r
1,0.110852,0.581068,1.124203,1.724269,t


In [13]:
def g1(df):
    mask = []
    for _, row in df.iterrows():
        if row["key"] in {"a", "b", "c", "d", "e"}:
            mask.append(True)
        else:
            mask.append(False)
    r = df[mask]
    return r

In [14]:
%%time
g1(df)

Wall time: 3min 21s


,col0,col1,col2,col3,key
7,0.589930,-0.320719,0.831536,2.555476,a
12,-0.466565,-0.467890,0.543636,0.442092,c
13,-0.064274,1.430865,-1.012040,1.418107,e
17,1.259945,-0.203664,-0.819146,2.202384,c
35,0.628336,0.196329,0.743559,2.295062,b
...,...,...,...,...,...
1999974,0.729830,0.707624,0.111283,-0.784053,b
1999976,0.373706,-1.336258,-0.525003,0.171614,c
1999979,0.731129,-0.906252,-0.731687,-0.508199,d
1999989,-0.428265,-1.128659,0.991428,-0.077673,a


In [15]:
%lprun -f g1 g1(df.head(20_000))

In [16]:
def g2(df):
    mask = df["key"].isin({"a", "b", "c", "d", "e"})
    return df[mask]

In [17]:
%%time
g2(df)

Wall time: 530 ms


,col0,col1,col2,col3,key
7,0.589930,-0.320719,0.831536,2.555476,a
12,-0.466565,-0.467890,0.543636,0.442092,c
13,-0.064274,1.430865,-1.012040,1.418107,e
17,1.259945,-0.203664,-0.819146,2.202384,c
35,0.628336,0.196329,0.743559,2.295062,b
...,...,...,...,...,...
1999974,0.729830,0.707624,0.111283,-0.784053,b
1999976,0.373706,-1.336258,-0.525003,0.171614,c
1999979,0.731129,-0.906252,-0.731687,-0.508199,d
1999989,-0.428265,-1.128659,0.991428,-0.077673,a


## Лабораторная работа 3

In [18]:
# !pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [19]:
import pandas as pd

In [20]:
recipes = pd.read_csv('recipes_sample.csv')
reviews = pd.read_csv('reviews_sample.csv')
reviews['date'] = pd.to_datetime(reviews['date']) #str to datetime format

In [21]:
def mean_rating_A(df):
    total = 0
    count = 0
    for _, row in df.iterrows():
        if row['date'].year == 2010:
            total += row['rating']
            count += 1
    return total/count
mean_rating_A(reviews)

4.4544402182900615

In [22]:
reviews_2010 = reviews[reviews["date"].dt.year == 2010]

In [23]:
def mean_rating_B(df):
    total = 0
    count = 0
    for _, row in df.iterrows():
        total += row["rating"]
        count += 1
    return total / count
mean_rating_B(reviews_2010)

4.4544402182900615

In [24]:
def mean_rating_C(df):
    return df['rating'].mean()
mean_rating_C(reviews_2010)

4.4544402182900615

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [25]:
!pip install line_profiler

In [26]:
from line_profiler import LineProfiler
lp = LineProfiler()
lp_wrapper_A = lp(mean_rating_A)
lp_wrapper_B = lp(mean_rating_B)
lp_wrapper_C = lp(mean_rating_C)

lp_wrapper_A(reviews)
lp_wrapper_B(reviews_2010)
lp_wrapper_C(reviews_2010)
lp.print_stats()

Timer unit: 1e-07 s

Total time: 0.0002989 s
File: C:\Users\User\AppData\Local\Temp\ipykernel_15604\1892587040.py
Function: mean_rating_C at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def mean_rating_C(df):
     2         1       2989.0   2989.0    100.0      return df['rating'].mean()

Total time: 39.2074 s
File: C:\Users\User\AppData\Local\Temp\ipykernel_15604\599982591.py
Function: mean_rating_A at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def mean_rating_A(df):
     2         1         48.0     48.0      0.0      total = 0
     3         1         25.0     25.0      0.0      count = 0
     4    126696  326661048.0   2578.3     83.3      for _, row in df.iterrows():
     5    114602   59829886.0    522.1     15.3          if row['date'].year == 2010:
     6     12094    5415106.0    447.8      1.4              total += row['rating

можем заметить, что фунцкия A выполняется дольше всего

In [27]:
#оптимизируем функцию Б
def mean_rating_B2(df):
    total = reviews_2010["rating"].sum()
    count = reviews_2010["rating"].count()
    return total/count
mean_rating_B2(reviews_2010)

4.4544402182900615

In [28]:
lp.print_stats()

Timer unit: 1e-07 s

Total time: 0.0002989 s
File: C:\Users\User\AppData\Local\Temp\ipykernel_15604\1892587040.py
Function: mean_rating_C at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def mean_rating_C(df):
     2         1       2989.0   2989.0    100.0      return df['rating'].mean()

Total time: 39.2074 s
File: C:\Users\User\AppData\Local\Temp\ipykernel_15604\599982591.py
Function: mean_rating_A at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def mean_rating_A(df):
     2         1         48.0     48.0      0.0      total = 0
     3         1         25.0     25.0      0.0      count = 0
     4    126696  326661048.0   2578.3     83.3      for _, row in df.iterrows():
     5    114602   59829886.0    522.1     15.3          if row['date'].year == 2010:
     6     12094    5415106.0    447.8      1.4              total += row['rating

как можем заметить, новвя функция В2(Total time: 0.0013574s) работает быстрее В(Total time: 0.764237s)

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [29]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [30]:
from collections import Counter

def get_word_reviews_count_optimized(df):
    words_counts = Counter()
    df_without_na = df.dropna(subset=["review"])

    def update_word_counts(row):
        words = set(row)
        words_counts.update(words)
    
    pattern = r"[^A-Za-z\s]"
    df_without_na['review'].str.replace(pattern, "").str.lower().str.split(" ").apply(update_word_counts)


    return dict(words_counts.items())

In [31]:
lp_get_word_reviews_count = lp(get_word_reviews_count)
lp_get_word_reviews_count(reviews)
lp_get_word_reviews_count_optimized = lp(get_word_reviews_count_optimized)
lp_get_word_reviews_count_optimized(reviews)
lp.print_stats()

C:\Users\User\AppData\Local\Temp\ipykernel_15604\2638786537.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df_without_na['review'].str.replace(pattern, "").str.lower().str.split(" ").apply(update_word_counts)


Timer unit: 1e-07 s

Total time: 0.0002989 s
File: C:\Users\User\AppData\Local\Temp\ipykernel_15604\1892587040.py
Function: mean_rating_C at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def mean_rating_C(df):
     2         1       2989.0   2989.0    100.0      return df['rating'].mean()

Total time: 10.4442 s
File: C:\Users\User\AppData\Local\Temp\ipykernel_15604\2638786537.py
Function: get_word_reviews_count_optimized at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def get_word_reviews_count_optimized(df):
     4         1        122.0    122.0      0.0      words_counts = Counter()
     5         1     418381.0 418381.0      0.4      df_without_na = df.dropna(subset=["review"])
     6                                           
     7         1         28.0     28.0      0.0      def update_word_counts(row):
     8                      

Как можем заметить оптимизированный код выполняпется за Total time: 2.84945 s, когда изначальный код выполняется за Total time: 32.4375 s 

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [38]:
#Без использования векторизованных операций и методов массивов numpy и без использования numba:
def MAPE_A(df):
    ratings = [r for r in df['rating'] if r != 0]
    mean_rating = sum(ratings) / len(ratings)
    abs_diff = 0
    for r in ratings:
        abs_diff += abs(r - mean_rating)
    return abs_diff / len(ratings) * 100
MAPE(reviews)

51.64167955430349

In [ ]:
#Без использования векторизованных операций и методов массивов numpy, но с использованием numba
import numba 

@numba.jit(nopython=True)
def MAPE(reviews):
    ratings = [r for r in reviews['rating'] if r != 0]
    mean_rating = sum(ratings) / len(ratings)
    abs_diff = 0
    for r in ratings:
        abs_diff += abs(r - mean_rating)
    return abs_diff / len(ratings) * 100
MAPE(reviews)

In [47]:
#С использованием векторизованных операций и методов массивов numpy, но без использования numba:

import numpy as np

def MAPE_C(reviews):
    ratings = np.array([r for r in reviews['rating'] if r != 0])
    mean_rating = np.mean(ratings)
    abs_diff = np.sum(np.abs(ratings - mean_rating))
    return abs_diff / len(ratings) * 100
MAPE_C(reviews)

51.64167955432964